# Прогнозирование на основе обученной модели классификации клиентов ЭДО 
# (Prediction based on a trained DMS client classification model)

### 1. Подготовка и обработка датасета (Preparation and preprocessing of dataset)

In [1]:
# standart library block 
import pandas as pd
import numpy as np
import datetime

In [2]:
# Чтение файла с данными (Reading data file)
dd_data = pd.read_excel("../DD_емкость КЭ.xlsx", sheet_name='Основная OBS', dtype = {'inn': object, 'kpp': object, 'okved': object,\
                                                                               'CLC': object, 'cnt_Kontur_product': object})

In [3]:
# Информация о датасете (Info about dataset)
dd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36801 entries, 0 to 36800
Data columns (total 63 columns):
inn                               36801 non-null object
kpp                               36801 non-null object
otrasl                            36801 non-null float64
segment                           36801 non-null float64
okved                             36801 non-null object
InvitesOut                        36801 non-null int64
InvitesOutAccepted                36801 non-null int64
InvitesIn                         36801 non-null int64
InvitesInAccepted                 36801 non-null int64
InvitesInALLTime                  36801 non-null int64
InvitesNoInYear                   36801 non-null int64
SearchesOut                       36801 non-null int64
SearchesIn                        36801 non-null int64
s1                                36801 non-null int64
trafficTop                        36801 non-null int64
v5portrait                        36801 non-null int64
top_

In [4]:
# Преобразование данных в числовой формат. Замена значений NaN на 0.
pd.set_option('display.max_rows', 63)
#dd_data = dd_data.apply(pd.to_numeric)
#dd_data.fillna(0) #преобразование значений Nan в 0
dd_data.tail().T

,36796,36797,36798,36799,36800
inn,2352047318,7710114301,9105001872,9102161211,3435312642
kpp,235201001,231943001,910501001,910201001,343501001
otrasl,0.03216,0.03129,0.02497,0.01151,0.11472
segment,0.02697,0.00757,0.0159,0.00693,0.04751
okved,56.1,86.1,84.1,47.7,43.2
InvitesOut,0,0,0,0,0
InvitesOutAccepted,0,0,0,0,0
InvitesIn,1,0,0,0,0
InvitesInAccepted,12,0,0,0,0
InvitesInALLTime,12,0,0,0,1


In [5]:
# Размер датасета (Dataset size)
dd_data.shape

(36801, 63)

### 2. Прогнозирование на основе обученной модели (Prediction based on a trained model)

In [6]:
# standart library block
import numpy as np
import pandas as pd
import pickle

# learning library block
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

from catboost import CatBoostClassifier
from catboost import cv
from catboost.datasets import amazon
from catboost.datasets import epsilon

#### Подготовка датасета (Prepared dataset)

In [7]:
# Определение target, feature. Распределение данных на тренировочную и тестовую выборки. 
# (Purpose of "target", "feature". Distribution of data on training and test samples.)
z=dd_data['inn'].copy()
X=dd_data.drop(['DD_service', 'inn','kpp', 'SearchesOutHalfAYear', 'SearchesInHalfAYear', 'Corp', 'DD_bill_date',\
                'DD_bill_sum', 'DD_pay_date', 'DD_pay_sum', 'DD_since', 'DD_upto', 'CLC',  'DD_service'], axis=1)

In [8]:
# Стандартизация значений переменных (Standardization of features values)
scaler = StandardScaler()
scaler.fit_transform(X)
X_scaled = scaler.transform(X)

C:\Users\voropaev.om\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\voropaev.om\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\voropaev.om\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


#### Прогнозирование - Модель RandomForest ver.1 (Prediction - RandomForest ver.1)

In [9]:
model_clf = RandomForestClassifier() # выбираем тип модели (choose the type of model)
model_clf = joblib.load('../2. Trained models/dd_clf_ver.2.pkl') # загружаем обученную модель (load trained model)

dd_pred_clf_train = model_clf.predict_proba(X)[:,1] # предсказание для рабочего датасета (prediction for work sample)

#### Прогнозирование - Модель GradientBoostingClassifier  ver.1 (Prediction - GradientBoostingClassifier  ver.1)

In [10]:
model_gbclf = GradientBoostingClassifier() # выбираем тип модели (choose the type of model)
model_gbclf = joblib.load('../2. Trained models/dd_gbclf_ver.2.pkl') # загружаем обученную модель (load trained model)

dd_pred_gbclf_train = model_gbclf.predict_proba(X)[:,1] # предсказание для рабочего датасета (prediction for work sample)

#### Прогнозирование - Модель CatBoost ver.1 (Prediction - Model CatBoost ver.1)

In [11]:
model = CatBoostClassifier() # выбираем тип модели (choose the type of model)
model.load_model('../2. Trained models/dd_cat_ver.2.cbm') # загружаем обученную модель (load trained model)

dd_pred_cat_train = model.predict_proba(data=X)[:,1] # предсказание для рабочего датасета (prediction for work sample)

#### Сохранение результатов

In [12]:
df1 = pd.DataFrame(z)

In [13]:
# Сохранение полученных резульатов модель RandomForest (RandomForest save the results)
df2_1 = pd.DataFrame(dd_pred_clf_train)
df2_1.columns=['Predictions']
res_clf = pd.concat([df1, df2_1.reindex(df1.index)], axis=1)
predict_results_clf = pd.DataFrame(res_clf).to_csv('../3. Results prediction/prediction_dd_clf_model_ver.2_KE.csv',\
                                                   index=False, sep=';')

In [14]:
# Сохранение полученных резульатов модель GradientBoosting (GradientBoosting save the results)
df2_2 = pd.DataFrame(dd_pred_gbclf_train)
df2_2.columns=['Predictions']
res_gbclf = pd.concat([df1, df2_2.reindex(df1.index)], axis=1)
predict_results_gbclf = pd.DataFrame(res_gbclf).to_csv('../3. Results prediction/prediction_dd_gbclf_model_ver.2_KE.csv',\
                                                       index=False, sep=';')

In [15]:
# Сохранение полученных резульатов модель CatBoost (CatBoost save the results)
df2_3 = pd.DataFrame(dd_pred_cat_train)
df2_3.columns=['Predictions']
#res_cat = pd.concat([df1, df2_3.reindex(df1.index)], axis=1)
predict_results_cat = pd.DataFrame(df2_3).to_csv('../3. Results prediction/prediction_dd_cat_model_ver.2_KE.csv',\
                                                 index=True, sep=';')